In [37]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

In [38]:
# armamos un data frame para usarlo como ejemplo de S&OP 500
tt_clients_data = {
        'id_pager': ['1','2','3','4','5'],
        'numero_telefono': ['1','2','3','4','5'],
        'codigo_de_area': ['11','11','11','22','22'],
        'fecha_creacion_cuenta': ['1983-01-01','1984-01-01','1984-01-01','1985-01-01','1986-01-01'],
        'nombre_cliente': ['Okabe', 'Kurisu', 'Mayuri','Suzuha', 'Daru'],
        'region': ['Akihabara', 'Akihabara', 'Akihabara', 'Kyoto', 'Kyoto'],
        'categoria_cliente': ['1','1','2','2','1']
}

tt_clients = pd.DataFrame(tt_clients_data, columns = tt_clients_data.keys())

In [39]:
# armamos un data frame para usarlo como ejemplo de S&OP 500
tt_msgs_data = {
        'anio': ['2020','2020','2020','2020','2020','2020','2020','2020','2020','2020'],
        'mes': ['3','3','3','3','4','4','4','4','4','4'],
        'dia': ['1','2','3','11','12','13','11','22','23','22'],
        'minutos': ['1','2','3','11','12','13','11','22','23','22'],
        'segundos': ['1','2','3','11','12','13','11','22','23','22'],
        'id_pager': ['2','1','2','1','5','1','2','1','2','1'],
        'mensaje': ["I'm the great mad scientist, Hououin Kyoma", "I am a scientist, I have to act on my own theory.", 
                    "It's an intellectual drink for the chosen ones", "People's feelings are memories that transcend time.",
                    "El Psy Kongroo", "Something must be wrong for you to use my actual name.",
                    "It's the choice of Steins Gate.", "You've pretty much figured it all out by now, right?",
                    "Sorry, The Zombie~~", "There was a scientific rationale for that!"],
        'numero_origen': ['1','2','1','2','1','1','2','1','2','1']
}

tt_msgs = pd.DataFrame(tt_msgs_data, columns = tt_msgs_data.keys())

 ¿Cuál es la región que aún tiene activos la mayor cantidad de pagers, entendiendo como activo aquellos que recibieron por lo menos un mensaje en el último mes?

In [40]:
tt_clients.head()

,id_pager,numero_telefono,codigo_de_area,fecha_creacion_cuenta,nombre_cliente,region,categoria_cliente
0,1,1,11,1983-01-01,Okabe,Akihabara,1
1,2,2,11,1984-01-01,Kurisu,Akihabara,1
2,3,3,11,1984-01-01,Mayuri,Akihabara,2
3,4,4,22,1985-01-01,Suzuha,Kyoto,2
4,5,5,22,1986-01-01,Daru,Kyoto,1


In [44]:
tt_msgs

,anio,mes,dia,minutos,segundos,id_pager,mensaje,numero_origen
0,2020,3,1,1,1,2,"I'm the great mad scientist, Hououin Kyoma",1
1,2020,3,2,2,2,1,"I am a scientist, I have to act on my own theory.",2
2,2020,3,3,3,3,2,It's an intellectual drink for the chosen ones,1
3,2020,3,11,11,11,1,People's feelings are memories that transcend ...,2
4,2020,4,12,12,12,5,El Psy Kongroo,1
5,2020,4,13,13,13,1,Something must be wrong for you to use my actu...,1
6,2020,4,11,11,11,2,It's the choice of Steins Gate.,2
7,2020,4,22,22,22,1,"You've pretty much figured it all out by now, ...",1
8,2020,4,23,23,23,2,"Sorry, The Zombie~~",2
9,2020,4,22,22,22,1,There was a scientific rationale for that!,1


In [46]:
#Obtengo los pagers con mensajes recibidos en el ultimo mes.

tt_msgs_activos = tt_msgs.loc[(tt_msgs['mes']=='4') & (tt_msgs['anio']=='2020')]
pagers_activos = tt_msgs_activos.groupby('id_pager').agg({'mensaje':'count'}).rename(columns={'mensaje':'cantidad_mensajes'})
pagers_activos = pagers_activos.reset_index()
pagers_activos = pagers_activos.drop('cantidad_mensajes', axis='columns')
clientes_activos = tt_clients.merge(pagers_activos, on='id_pager')
clientes_activos

,id_pager,numero_telefono,codigo_de_area,fecha_creacion_cuenta,nombre_cliente,region,categoria_cliente
0,1,1,11,1983-01-01,Okabe,Akihabara,1
1,2,2,11,1984-01-01,Kurisu,Akihabara,1
2,5,5,22,1986-01-01,Daru,Kyoto,1


In [48]:
#Conociendo los activos agrupo por región y veo cual es la región con mas activos

clientes_activos_por_region = clientes_activos.groupby('region').agg({'id_pager':'count'}).rename(columns={'id_pager':'cant_activos'})
clientes_activos_por_region

,cant_activos
region,
Akihabara,2
Kyoto,1


In [52]:
#Conociendo los activos por región puedo conocer la región que mas clientes activos posee.

clientes_activos_por_region[clientes_activos_por_region['cant_activos']==clientes_activos_por_region['cant_activos'].max()]

,cant_activos
region,
Akihabara,2


¿Cuál es el porcentaje de pagers activos que solamente reciben mensajes únicamente de una persona (es decir, siempre desde el mismo número de origen)?

In [53]:
tt_msgs_activos

,anio,mes,dia,minutos,segundos,id_pager,mensaje,numero_origen
4,2020,4,12,12,12,5,El Psy Kongroo,1
5,2020,4,13,13,13,1,Something must be wrong for you to use my actu...,1
6,2020,4,11,11,11,2,It's the choice of Steins Gate.,2
7,2020,4,22,22,22,1,"You've pretty much figured it all out by now, ...",1
8,2020,4,23,23,23,2,"Sorry, The Zombie~~",2
9,2020,4,22,22,22,1,There was a scientific rationale for that!,1


In [62]:
nums_que_reciben_msg = tt_msgs_activos.groupby('id_pager').agg({'numero_origen':'nunique'}).rename\
    (columns={'numero_origen': 'cant_nums_unicos_que_envian_msg'})
p_activos_unicos_1_persona = nums_que_reciben_msg[nums_que_reciben_msg['cant_nums_unicos_que_envian_msg']==1]
porcentaje = len(p_activos_unicos_1_persona)*100/len(nums_que_reciben_msg)

print ('El porcentaje de pagers que reciben de una única persona es: ' + str(porcentaje) + '%.')


El porcentaje de pagers que reciben de una única persona es: 100.0%.
